In [1]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn import metrics
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import *
from scipy.stats import itemfreq
from sklearn.metrics import accuracy_score

In [2]:
fold_1 = np.load('./unshape/fold1_unshape.npy')
target_1 = np.load('./unshape/fold1_class_unshape.npy')

In [3]:
fold_2 = np.load('./unshape/fold2_unshape.npy')
target_2 = np.load('./unshape/fold2_class_unshape.npy')

In [4]:
fold_3 = np.load('./unshape/fold3_unshape.npy')
target_3 = np.load('./unshape/fold3_class_unshape.npy')

In [5]:
fold_4 = np.load('./unshape/fold4_unshape.npy')
target_4 = np.load('./unshape/fold4_class_unshape.npy')

In [6]:
train = pd.DataFrame(np.concatenate((fold_1,fold_2,fold_3), axis=0))
target = np.concatenate((target_1,target_2,target_3), axis=0)

In [7]:
net = MLPClassifier(activation='relu', solver='adam', learning_rate_init=0.003, 
                    alpha=2e-5, hidden_layer_sizes=(180,120,60), random_state=1)

scaler = RobustScaler()

In [8]:
scaler.fit(train)
train_norm = scaler.transform(train)

In [9]:
net.fit(train_norm, target)

MLPClassifier(activation='relu', alpha=2e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(180, 120, 60), learning_rate='constant',
       learning_rate_init=0.003, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,
       solver='adam', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)

In [10]:
with open('./fold4_evaluate_revise.txt') as f:
    lines = f.readlines()

In [11]:
class2num = {"class":     {"bus": 0, "cafe/restaurant": 1, "car": 2, "city_center": 3, "forest_path": 4, 
                              "grocery_store": 5, "home": 6, "beach": 7, "library": 8, "metro_station": 9, 
                              "office": 10, "residential_area": 11, "train": 12, "tram": 13, "park": 14}}

In [12]:
y_true = np.empty(shape=[0,1])
y_pred = np.empty(shape=[0,1])

In [13]:
for line in lines:
    description = line.split('\t')
    
    ## decoding class to string
    description[1] = description[1].replace('\n', '')
    
    ## convert string to numerical value
    description[1] = class2num["class"][description[1]]
    
    ## append true value
    y_true = np.append(y_true, description[1])
    
    ## import data
    try:
        data = np.load('./DCASE2017_development_set/mfcc_numpy/' + description[0])
    except:
        print("ERROR")
        
    ## convert to data frame
    df = pd.DataFrame(data)
    df = df.T
    
    ## prediction
    df = scaler.transform(df)
    predicts = net.predict(df)
    
    ## voting
    pred_cnt = itemfreq(predicts)
    voting_result = pred_cnt[pred_cnt[:,1].argsort()][-1,0]
    
    ## append voting result
    y_pred = np.append(y_pred, voting_result)

In [14]:
acc = accuracy_score(y_true, y_pred)

In [15]:
acc

0.7051282051282052

In [17]:
train_final = np.append(train, fold_4, axis=0)

target_final = np.append(target, target_4, axis=0)

In [19]:
net_final = MLPClassifier(activation='relu', solver='adam', learning_rate_init=0.003, 
                    alpha=2e-5, hidden_layer_sizes=(180,120,60), random_state=1)
scaler_final = RobustScaler()
scaler_final.fit(train_final)
train_norm_final = scaler_final.transform(train_final)


In [20]:
net_final.fit(train_norm_final, target_final)

MLPClassifier(activation='relu', alpha=2e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(180, 120, 60), learning_rate='constant',
       learning_rate_init=0.003, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,
       solver='adam', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)

need to be changed to the right format 

In [31]:
for i in range(1,1620):
    evaluation_example = np.load('./mfcc_d/'+str(i)+'.npy')
    df = pd.DataFrame(evaluation_example)
    df = df.T
    df = scaler_final.transform(df)
    predicts = net_final.predict(df)
    pred_cnt = itemfreq(predicts)
    voting_result = pred_cnt[pred_cnt[:,1].argsort()][-1,0]
    with open('results.txt', 'a') as the_file:
        the_file.write(str(voting_result)+"\n")